In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common import NoSuchElementException
from selenium.common import TimeoutException
import numpy as np
from selenium.webdriver import ActionChains
import pandas as pd
import re

In [ ]:
# Initialize a variable to store the text of the last review from the previous iteration
previous_last_review_text = None
return_to_first_review()

while True:
    short_wait.until(EC.invisibility_of_element_located(loading_text))

    review_elements = driver.find_elements(By.XPATH, "//*[@data-hveid='2']/following-sibling::*//div[@class='OA1nbd']")

    if not review_elements:
        break  # Break if no reviews are found

    # Get the text of the last review element
    current_last_review_text = review_elements[-1].text.strip()

    # Check if the last review element is empty
    if not current_last_review_text:
        print("Empty review encountered.")
        break  # Break if the last review is empty
    short_wait.until(EC.invisibility_of_element_located(loading_text))
    last_element = review_elements[-1]
    last_element.location_once_scrolled_into_view
    #time.sleep(2)  # Waiting for potential loading of new reviews
    #short_wait.until(EC.visibility_of_element_located(loading_text))
    short_wait.until(EC.invisibility_of_element_located(loading_text))

    # Check if the last non-empty review text is the same as in the previous iteration
    if current_last_review_text == previous_last_review_text:
        total_reviews_with_text = [elem for elem in review_elements if elem.text.strip()]
        print(len(total_reviews_with_text), "reviews found.")
        break  # Break the loop if we are seeing the same last non-empty review as before

    previous_last_review_text = current_last_review_text  # Update the last review text for the next iteration


In [ ]:
def return_to_first_review():
    first_review = driver.find_element(By.XPATH,
                                       "//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[1]//div[@class='OA1nbd']")
    first_review.location_once_scrolled_into_view
    driver.execute_script("arguments[0].scrollIntoView();", first_review)


In [ ]:
# Fetch all review elements
review_elements_xpath = "//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div//div[@class='OA1nbd']"
review_elements = driver.find_elements(By.XPATH, review_elements_xpath)

# First pass: Click all 'Translate' buttons
for i in range(1, len(review_elements) + 1):
    translate_button_xpath = f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]/div[3]//button[contains(@class, 'qGGAec') and .//span[contains(text(), 'Translated by Google')]]"
    translate_buttons = driver.find_elements(By.XPATH, translate_button_xpath)
    if translate_buttons:
        translate_buttons[0].click()
        #time.sleep(1)  # Adjust timing as necessary

In [ ]:
# Second pass: Click all 'More' buttons
for i in range(1, len(review_elements) + 1):
    more_button_xpath = f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]//div[@class='OA1nbd']//a[@role='button' and contains(text(), 'More')]"
    more_buttons = driver.find_elements(By.XPATH, more_button_xpath)
    if more_buttons and more_buttons[0].is_displayed():
        driver.execute_script("arguments[0].scrollIntoView();", more_buttons[0])
        try:
            more_buttons[0].click()
        except Exception as e:
            print(f"Skipping a non-interactable 'More' button at index {i}: {e}")


In [ ]:
def extract_rating(aria_label):
    match = re.search(r"Rated (\d+(\.\d+)?) out of 5", aria_label)
    return float(match.group(1)) if match else None

In [ ]:
reviews_df = pd.DataFrame(columns=['Review', 'Rating'])
reviews_df.reset_index(drop=True, inplace=True)
return_to_first_review()
i = 1
count_non_empty_reviews = 0

while count_non_empty_reviews <= len(total_reviews_with_text):
    try:
        review_xpath = f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]//div[@class='OA1nbd']"
        rating_xpath = f"//div[@data-attrid='kc:/local:all reviews']//div[3]/div/div[{i}]//div[contains(@aria-label, 'Rated') and contains(@role, 'img')]"

        review_element = driver.find_element(By.XPATH, review_xpath)
        rating_element = driver.find_element(By.XPATH, rating_xpath)

        review_text = review_element.text.strip()
        if review_text:
            review_rating = extract_rating(rating_element.get_attribute('aria-label'))
            reviews_df.loc[len(reviews_df)] = [review_text, review_rating]
            count_non_empty_reviews += 1

    except NoSuchElementException:
        print("No more reviews found.")

    i += 1

print(reviews_df.head())

reviews_df.to_csv('reviews_and_ratings.csv', index=False)